In [1]:
# !pip install yandex_geocoder

I have a table with addresses of buildings and I want to get the coordinates of these buildings. For example, I use [`yandex_geocoder`](!https://yandex.com/dev/maps/geocoder/), you can choose any other. Yandex is better for my hometown.

In [2]:
import pandas as pd
import numpy as np

from yandex_geocoder import Client


In [3]:
your_api_code = "code"


In [4]:
client = Client(your_api_code)


In [5]:
client.coordinates(address='Хабаровск 60 октября 150')


(Decimal('135.113751'), Decimal('48.478187'))

In [6]:
data = pd.read_csv('../data/reestr.csv', sep=';')


In [7]:
data.head()


,id,region_id,area_id,city_id,street_id,shortname_region,formalname_region,shortname_area,formalname_area,shortname_city,...,electrical_entries_count,heating_type,hot_water_type,cold_water_type,sewerage_type,sewerage_cesspools_volume,gas_type,ventilation_type,firefighting_type,drainage_type
0,7114412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,Квартирное отопление (квартирный котел),"Автономная котельная (крышная, встроенно-прист...",Центральное,Центральное,"0,00",Центральное,Вытяжная вентиляция,Отсутствует,Отсутствует
1,6732563,88cd27e2-6a8a-4421-9718-719a28a0a088,dab8ebdf-5f74-480a-809e-fca52235ceef,b38da249-f672-41e6-a641-f6f77f59208b,8638ec45-626e-48a8-a3f7-55aa2fa3c2a3,обл,Нижегородская,р-н,Ардатовский,рп,...,NaN,Не заполнено,Не заполнено,Не заполнено,Не заполнено,NaN,Автономное,Не заполнено,Не заполнено,Не заполнено
2,6732551,88cd27e2-6a8a-4421-9718-719a28a0a088,dab8ebdf-5f74-480a-809e-fca52235ceef,b38da249-f672-41e6-a641-f6f77f59208b,8638ec45-626e-48a8-a3f7-55aa2fa3c2a3,обл,Нижегородская,р-н,Ардатовский,рп,...,NaN,Не заполнено,Не заполнено,Не заполнено,Не заполнено,NaN,Автономное,Не заполнено,Не заполнено,Не заполнено
3,7728846,88cd27e2-6a8a-4421-9718-719a28a0a088,dab8ebdf-5f74-480a-809e-fca52235ceef,b38da249-f672-41e6-a641-f6f77f59208b,8fc4087c-f909-4d51-b18e-f641ba138175,обл,Нижегородская,р-н,Ардатовский,рп,...,NaN,Не заполнено,Не заполнено,Не заполнено,Не заполнено,NaN,Автономное,Не заполнено,Не заполнено,Не заполнено
4,7617624,88cd27e2-6a8a-4421-9718-719a28a0a088,dab8ebdf-5f74-480a-809e-fca52235ceef,b38da249-f672-41e6-a641-f6f77f59208b,8fc4087c-f909-4d51-b18e-f641ba138175,обл,Нижегородская,р-н,Ардатовский,рп,...,1.0,Квартирное отопление (квартирный котел),Квартирное (квартирный котел),Центральное,Автономное,"10,80",Центральное,Вытяжная вентиляция,Отсутствует,Отсутствует


In [8]:
df = data[['address', 'floor_count_max', 'built_year', 'is_alarm']]
df = df.dropna().reset_index(drop=True)
df


,address,floor_count_max,built_year,is_alarm
0,д. г Навашино ул Соболева д.12,2.0,1954.0,Нет
1,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1917.0,Нет
2,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1977.0,Нет
3,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1977.0,Нет
4,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1970.0,Нет
...,...,...,...,...
22517,"обл. Нижегородская, г. Шахунья, ул. Энгельса, ...",3.0,1960.0,Нет
22518,"обл. Нижегородская, г. Шахунья, ул. Энгельса, ...",2.0,1975.0,Нет
22519,"обл. Нижегородская, г. Шахунья, ул. Энгельса, ...",2.0,1981.0,Нет
22520,"обл. Нижегородская, г. Шахунья, ш. Яранское, д...",2.0,1976.0,Нет


Geocoding is a long process, our task is not about that. For example, I took the first 10 rows.

In [9]:
df = df[:10].copy()

df['Lat'] = None
df['Lon'] = None


In [10]:
for i, row in df.iterrows():
    lon, lat = client.coordinates(row['address'])

    df.at[i, 'Lat'] = lat
    df.at[i, 'Lon'] = lon


In [11]:
df

,address,floor_count_max,built_year,is_alarm,Lat,Lon
0,д. г Навашино ул Соболева д.12,2.0,1954.0,Нет,51.762101,19.513708
1,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1917.0,Нет,55.235595,43.095472
2,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1977.0,Нет,55.234774,43.09496
3,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1977.0,Нет,55.233059,43.09664
4,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1970.0,Нет,55.232489,43.094367
5,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1976.0,Нет,55.232905,43.095364
6,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1976.0,Нет,55.232905,43.095364
7,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1970.0,Нет,55.231914,43.094403
8,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1977.0,Нет,55.232459,43.095355
9,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,1971.0,Нет,55.231509,43.094394


For the rest of the data, I used geocoding with more loyal terms of use.

I can use geocoding on other free services, and I can send problematic cases to [`yandex_geocoder`](!https://yandex.com/dev/maps/geocoder/), but again, this is not our task. I don't check the quality of geocoding (we can see, that 1st row has coordinates from another region). But, this does not affect the result.

The separator character ";" is used to avoid problems with commas and dots in the address or coordinates.

In [16]:
geocoded_data = pd.read_csv('../data/geocoded_data.csv', sep=';')
geocoded_data


,address,floor_count_max,is_alarm,built_year,Lat,Lon
0,д. г Навашино ул Соболева д.12,2.0,Нет,1954.0,52.013247,23.099710
1,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,Нет,1917.0,55.235595,43.095463
2,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,Нет,1977.0,55.234774,43.094960
3,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,Нет,1977.0,55.233054,43.096613
4,"обл. Нижегородская, р-н. Ардатовский, рп. Арда...",2.0,Нет,1970.0,55.232484,43.094367
...,...,...,...,...,...,...
22516,"обл. Нижегородская, г. Шахунья, ул. Энгельса, ...",3.0,Нет,1960.0,57.680099,46.597824
22517,"обл. Нижегородская, г. Шахунья, ул. Энгельса, ...",2.0,Нет,1975.0,57.679579,46.596539
22518,"обл. Нижегородская, г. Шахунья, ул. Энгельса, ...",2.0,Нет,1981.0,57.677732,46.593018
22519,"обл. Нижегородская, г. Шахунья, ш. Яранское, д...",2.0,Нет,1976.0,57.659796,46.601300


In [13]:
geocoded_data['Lat'], geocoded_data['Lon'] = geocoded_data['Lon'], geocoded_data['Lat']


In [15]:
geocoded_data.to_csv('../data/geocoded_data.csv', sep=';', index=False)
